## Health Insurance Issuers by County--Republican states offer fewer options

Using the Bloomberg data from this story: https://www.bloomberg.com/graphics/2017-health-insurer-exits/ matched with population data from Census and the Governor's party affiliation from the NGA.

In [1]:
import pandas as pd
import config   # File that contains api key
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

#### Collect data

In [26]:
# bloomberg data on issuers by county
df = pd.read_csv('issuers_bloomberg_data.csv').query('year == 2017').set_index('year')
df = df.groupby('fips_county').issuer_name.count()

# Census and state governor data
df2 = pd.read_csv('census_pop.csv').set_index('state').join(
    pd.read_csv('nga_gov_list.csv').set_index('state')).dropna().reset_index().set_index('fips_county')
data = df2.join(df).dropna()

# Group issuer number
data['issuer_number'] = data['issuer_name']
d = {1: '1', 2: '2', 3: '3+', 4: '3+', 5: '3+', 6: '3+', 7: '3+', 8: '3+', 9: '3+', 10: '3+', 11: '3+', 12: '3+', 13: '3+'}
data = data.replace({'issuer_number': d})

# Latitude and longitude
ll = pd.read_table('2015_Gaz_counties_national.txt').rename(columns={'GEOID': 'fips_county'}).set_index('fips_county')
ll['INTPTLONG'] = ll.iloc[:,-1].rename('INTPTLONG')
ll = ll[['INTPTLAT', 'INTPTLONG']]
# Merge
data = data.join(ll)

# Move a few specific locations
data.loc[2013, 'INTPTLAT':'INTPTLONG'] = [55.3397, -160.4972]
data.loc[2016, 'INTPTLAT':'INTPTLONG'] = [53.8844, -166.5332]
data.loc[17097, 'INTPTLAT':'INTPTLONG'] = [42.3636, -87.8448]

# Save file
data.to_csv('clean_data.csv')

#### Basic calculations

In [27]:
pd.pivot_table(data, values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)

pop
gov_party   issuer_number           
Democratic  1               10714794
            2               24084228
            3+              92549769
Independent 1                 733845
Republican  1               40182082
            2               40763075
            3+             113395998

In [28]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)

pop
gov_party   issuer_number           
Democratic  1                2114291
            2               22537943
            3+              92549769
Independent 1                 733845
Republican  1               40182082
            2               40763075
            3+             113395998

In [29]:
pd.pivot_table(data, values='pop', index=['issuer_number'], aggfunc=sum)

,pop
issuer_number,
1,51630721
2,64847303
3+,205945767


In [30]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum).groupby(
    level=0).transform(lambda x: x/x.sum())

pop
gov_party   issuer_number          
Democratic  1              0.018040
            2              0.192300
            3+             0.789660
Independent 1              1.000000
Republican  1              0.206761
            2              0.209750
            3+             0.583489

In [31]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)

pop
gov_party   issuer_number           
Democratic  1                2114291
            2               22537943
            3+              92549769
Independent 1                 733845
Republican  1               40182082
            2               40763075
            3+             113395998

#### Plot data

In [10]:
df = pd.read_csv('clean_data.csv').set_index('fips_county')

# Keep counties with one issuer
df = df[df['issuer_number'] == '1']

# Generate text for hover tool
df['Governor'] = ['{} ({})'.format(row['gov_name'], row['gov_party'][0]) for index, row in df.iterrows()]
df['text'] = ['{}<br>Population: {:,}<br>Governor: {}'.format(
    row['county_name'], row['pop'], row['Governor']) for index, row in df.iterrows()]

# Calculate size
df['size'] = df['pop']/40000 + 4
df = df.sort_values('pop', ascending=False)
# Determint color
for index, row in df.iterrows():
    if row['gov_party'] == 'Republican':
        df.loc[index,'color'] = '#cc0000'
    if row['gov_party'] == 'Democratic':
        df.loc[index,'color'] = '#1C459C'
    if row['gov_party'] == 'Independent':
        df.loc[index,'color'] = '#B2B200'

In [11]:
trace1 = {
  "geo": "geo", 
  "hoverinfo": "text", 
  "lat": df['INTPTLAT'], 
  "lon": df['INTPTLONG'], 
  "marker": {
    "color": df['color'], 
    "line": {
      "color": "#fff", 
      "width": 1
    }, 
    "opacity": 0.5, 
    "size": df['size'], 
    "symbol": "circle"
  }, 
  "mode": "markers", 
  "name": "Counties with one health care exchange issuer", 
  "opacity": 1, 
  "showlegend": True, 
  "text": df['text'], 
  "type": "scattergeo", 
  "visible": True
}
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": 0.5, 
      "y": 0.99, 
      "align": "center", 
      "font": {"size": 28, "family": "PT Sans Narrow"}, 
      "showarrow": False, 
      "text": "<b>Counties with only one health insurance<br>provider on state exchange</b>", 
      "xref": "canvas", 
      "yref": "canvas"
    }, 
    {
      "x": 0.5, 
      "y": 0, 
      "align": "center", 
      "showarrow": False, 
      "font": {"size": 13, "family": "PT Sans Narrow"},
      "text": "Hover to see details. Color is state governor political party: blue is Democratic, red is Republican, <br>yellow is Independent. Circle size is population. <b>Sources:</b> Bloomberg compiled data from<br>Healthcare.gov and state exchanges; National Governors Association; U.S. Census Bureau.", 
      "xref": "canvas", 
      "yref": "canvas"
    }
  ], 
  "autosize": True, 
  "dragmode": "zoom", 
  "geo": {
    "landcolor": "rgb(212, 216, 220)",
    "lakecolor": "rgb( 138, 228, 255)",
    "bgcolor": "rgb(255, 252, 245)", 
    "countrycolor": "#808080", 
    "countrywidth": 0.7, 
    "domain": {
      "x": [-0.1, 1.1], 
      "y": [-0.1, 1.1]
    }, 
    "lataxis": {
      "dtick": 10, 
      "range": [22, 50], 
      "showgrid": False, 
      "tick0": 22
    }, 
    "lonaxis": {
      "dtick": 30, 
      "range": [-130, -60], 
      "showgrid": False, 
      "tick0": -130
    }, 
    "projection": {
      "scale": 1.03, 
      "type": "albers usa"
    }, 
    "resolution": 50, 
    "scope": "usa", 
    "showcountries": False, 
    "showlakes": True, 
    "showland": True, 
    "showrivers": False, 
    "showsubunits": True, 
    "subunitcolor": "#fff", 
    "subunitwidth": 0.7
  }, 
  "hidesources": False, 
  "hovermode": "closest", 
  "paper_bgcolor": "rgb(255, 252, 245)", 
  "plot_bgcolor": "rgb(255, 252, 245)", 
  "separators": ".,", 
  "showlegend": False, 
  "smith": False, 
  "margin": {
    "r": 0, 
    "t": 0, 
    "b": 0, 
    "l": 0,
    "pad": 0 
  },
}
fig = Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='HC_County')
py.iplot(fig, filename='HC_County')

In [ ]:
tot = 51630721.0
d_sh = 10714794.0/tot
r_sh = 40182082.0/tot
tot_nc = 43030218.0
d_sh_nc = 10714794.0/tot_nc
r_sh_nc = 40182082.0/tot_nc

In [ ]:
d_sh_nc